In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-energy-behavior-of-prosumers/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv
/kaggle/input/predict-energy-behavior-of-prosumers/public_timeseries_testing_util.py
/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/county_id_to_name_map.json
/kaggle/input/predict-energy-behavior-of-prosumers/train.csv
/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/sample_submission.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity

In [2]:
# 导入必要的库
import holidays
import os  # 操作系统相关功能
import sys  # 提供对 Python 解释器的访问
import gc  # 垃圾回收模块
import time  # 时间模块
import math  # 数学函数模块
import collections  # 集合模块
import psutil  # 进程和系统状态信息模块
import pickle  # 对象序列化和反序列化模块
from datetime import date, datetime, timedelta  # 日期和时间相关模块
from copy import deepcopy  # 复制对象模块
from tqdm import tqdm  # 进度条模块

import numpy as np  # 数组和矩阵操作库
import pandas as pd  # 数据分析库
import polars as pl  # 快速数据操作库

import matplotlib.pyplot as plt  # 绘图库
import seaborn as sns  # 数据可视化库
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf  # 时间序列分析模块

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error  # 均方误差指标
from sklearn.model_selection import GroupKFold, StratifiedKFold, KFold, TimeSeriesSplit  # 不同的交叉验证方法
from sklearn.compose import TransformedTargetRegressor  # 目标变换模块
from sklearn.ensemble import VotingRegressor  # 集成回归器
from sklearn.model_selection import cross_validate # 本地交叉验证

import lightgbm as lgb
from xgboost import XGBRegressor, plot_importance

import optuna  # 超参数优化框架'

In [3]:
# 计算某指标的前n个时间单元的平均值和方差
def calculate_mean_var(df: pd.DataFrame, n, column_name):
    mean = pd.Series()
    var = pd.Series()
    for i in range(len(df)):
        if i < n-1:
            mean[i] = np.mean(df[column_name].loc[0:i])
            var[i] = np.var(df[column_name].loc[0:i])
        else:
            mean[i] = np.mean(df[column_name].loc[i-n+1:i])
            var[i] = np.var(df[column_name].loc[i-n+1:i])
    df[column_name +'_former_' + str(n) + '_mean'] = mean
    df[column_name +'_former_' + str(n) + '_var'] = var
    
    return df

In [4]:
def add_holidays_as_binary_features(df: pd.DataFrame):
    estonian_holidays = holidays.country_holidays('EE', years=range(2021, 2025))
    estonian_holidays_set = {date.strftime('%Y-%m-%d') for date in estonian_holidays.keys()}
    # 获取年月日信息
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day

    # 根据年月日确定是否是公共假期
    days = df['year'].astype(str) + '-' + df['month'].astype(str).str.zfill(2) + '-' + df['day'].astype(str).str.zfill(2)
    df['country_holiday'] = days.isin(estonian_holidays_set) 

    df = df.drop(columns = ['year', 'month', 'day'])
    return df

In [5]:
def join_target_on_date(target ,target1 ,days, suffix):
    target1 = target[['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']].copy()
    target1['datetime'] += pd.to_timedelta(days, unit='D')
    new_target = target1.rename(columns={"target": f"target_{suffix}"})
    target = target.merge(new_target, left_on=["county", "is_business", "product_type", "is_consumption", "datetime"], right_on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="inner")
    return target

In [6]:
def feature_eng1(train, client, gas, electricity, fw, hw, locations, df_target):
    # 处理locations表
    locations = locations.dropna()
    locations = locations.drop_duplicates()
    locations = locations.reset_index(drop = True)
    locations['latitude'] = round(locations['latitude'],1)
    locations['longitude'] = round(locations['longitude'],1)

    # 将fw中的列标记为预测天气(_fw)
    fw['origin_datetime'] = pd.to_datetime(fw['origin_datetime'])
    fw['forecast_datetime'] = pd.to_datetime(fw['forecast_datetime'])
    fw = fw.rename(columns={'origin_datetime': 'origin_datetime_fw',
                                           'hours_ahead':'hours_ahead_fw',
                                           'temperature':'temperature_fw',
                                           'dewpoint':'dewpoint_fw',
                                           'cloudcover_high':'cloudcover_high_fw',
                                           'cloudcover_low':'cloudcover_low_fw',
                                           'cloudcover_mid':'cloudcover_mid_fw',
                                           'cloudcover_total':'cloudcover_total_fw',
                                            '10_metre_u_wind_component':'10_metre_u_wind_component_fw',
                                           '10_metre_v_wind_component':'10_metre_v_wind_component_fw',
                                           'direct_solar_radiation':'direct_solar_radiation_fw',
                                           'surface_solar_radiation_downwards':'surface_solar_radiation_downwards_fw',
                                           'snowfall':'snowfall_fw',
                                           'total_precipitation':'total_precipitation_fw'})
    # 将经纬度小数点位数统一
    fw['latitude'] = round(fw['latitude'],1)
    fw['longitude'] = round(fw['longitude'],1)
    
    # 合并预测天气和经纬度信息
    fw_loc = pd.merge(fw,locations,how="inner",left_on=['longitude','latitude'],right_on=['longitude','latitude'])
    
    # 删掉['latitude', 'longitude', 'origin_datetime_fw', 'county_name', 'hours_ahead_fw']这些与预测无关的列。
    # 若hours_ahead_fw删掉，原因是：在所有时间点，可用的天气预报数据只有一行（需要再次确定）。
        # 若hours_ahead_fw不删，原因是：做预测的时间与预测时间的时间差会影响预测准确性，进而影响用户行为。
    fw_loc = fw_loc.drop(columns = ['latitude', 'longitude', 'origin_datetime_fw', 'county_name']) 
    # 因为同一个城市占据多个经纬度点，多个经纬度点的数据求平均
    fw_loc_groupby = fw_loc.groupby(['county', 'forecast_datetime']).mean()
    fw_loc_groupby = fw_loc_groupby.reset_index()
    fw_loc_groupby['forecast_datetime'] = pd.to_datetime(fw_loc_groupby['forecast_datetime'])

    # 合并历史数据和位置信息
    hw_loc = pd.merge(hw, locations, how = 'inner', left_on=['longitude','latitude'],right_on=['longitude','latitude'])
    
    # 丢弃与预测无关的['datetime', 'longitude', 'latitude']
    hw_loc = hw_loc.drop(columns = [ 'longitude', 'latitude', 'county_name'])
    
    # 计算所有历史天气数据关于同一data_block_id和county的均值
    hw_loc_groupby_mean = hw_loc.groupby(['datetime', 'county']).mean()
    hw_loc_groupby_mean = hw_loc_groupby_mean.reset_index()
    hw_loc_groupby_mean = hw_loc_groupby_mean.rename(columns={
                                       'temperature':'temperature_hw_mean',
                                       'dewpoint':'dewpoint_hw_mean',
                                       'rain':'rain_hw_mean',
                                       'snowfall':'snowfall_hw_mean',
                                       'surface_pressure':'surface_pressure_hw_mean',
                                       'cloudcover_high':'cloudcover_high_hw_mean',
                                       'cloudcover_low':'cloudcover_low_hw_mean',
                                       'cloudcover_mid':'cloudcover_mid_hw_mean',
                                       'cloudcover_total':'cloudcover_total_hw_mean',
                                       'windspeed_10m':'windspeed_10m_hw_mean',
                                       'winddirection_10m':'winddirection_10m_hw_mean',
                                       'shortwave_radiation':'shortwave_radiation_hw_mean',
                                       'diffuse_radiation':'diffuse_radiation_hw_mean',
                                       'direct_solar_radiation':'direct_solar_radiation_hw_mean',
                                       })
    
    # 计算所有历史天气数据关于同一data_block_id和county的方差
    hw_loc_groupby_var = hw_loc.groupby(['datetime', 'county']).var()
    hw_loc_groupby_var = hw_loc_groupby_var.reset_index()
    hw_loc_groupby_var = hw_loc_groupby_var.rename(columns={
                                       'temperature':'temperature_hw_var',
                                       'dewpoint':'dewpoint_hw_var',
                                       'rain':'rain_hw_var',
                                       'snowfall':'snowfall_hw_var',
                                       'surface_pressure':'surface_pressure_hw_var',
                                       'cloudcover_high':'cloudcover_high_hw_var',
                                       'cloudcover_low':'cloudcover_low_hw_var',
                                       'cloudcover_mid':'cloudcover_mid_hw_var',
                                       'cloudcover_total':'cloudcover_total_hw_var',
                                       'windspeed_10m':'windspeed_10m_hw_var',
                                       'winddirection_10m':'winddirection_10m_hw_var',
                                       'shortwave_radiation':'shortwave_radiation_hw_var',
                                       'diffuse_radiation':'diffuse_radiation_hw_var',
                                       'direct_solar_radiation':'direct_solar_radiation_hw_var',
                                       })


    # 合并历史天气的均值和方差信息
    hw_loc_groupby = pd.merge(hw_loc_groupby_mean, hw_loc_groupby_var, how = 'inner', 
                            left_on = ['datetime', 'county'], right_on = ['datetime', 'county'])
    hw_loc_groupby['datetime'] = pd.to_datetime(hw_loc_groupby['datetime'])
    
    # 重命名 'forecast_date' 列为 'datetime'
    electricity.rename(columns={"forecast_date": "datetime"}, inplace=True)
    # 在 'datetime' 列上添加一天的时间间隔
    electricity['datetime'] =  pd.to_datetime(pd.to_datetime(electricity['datetime']) + pd.DateOffset(days=1))
    
    # 对于每一小时，计算electricity的前24小时均值和方差(要不要再计算下前一个周的？)
    electricity = calculate_mean_var(electricity, 24, 'euros_per_mwh')
    electricity = electricity.drop(columns = [ 'origin_date'])
    electricity = electricity.rename(columns={"euros_per_mwh": "euros_per_mwh_electric", 
                                        "euros_per_mwh_former_24_mean": "euros_per_mwh_former_24_mean_electric",
                                        "euros_per_mwh_former_24_var":"euros_per_mwh_former_24_var_electric"})
    
    
    
    gas['date'] =  pd.to_datetime(pd.to_datetime(gas['forecast_date']) + pd.Timedelta(days=1))
    # 对于每一天，计算gas价格的前7天均值和方差
    gas = calculate_mean_var(gas, 7, 'lowest_price_per_mwh')
    gas = calculate_mean_var(gas, 7, 'highest_price_per_mwh')
    
    # 将gas的时间列删除
    gas = gas.drop(columns = [ 'forecast_date','origin_date'])
    
    # 给汽油价格列重命名防止与电价价混淆
    gas = gas.rename(columns={"lowest_price_per_mwh": "lowest_price_per_mwh_gas", 
                          "highest_price_per_mwh": "highest_price_per_mwh_gas",
                          "lowest_price_per_mwh_former_7_mean_gas":"lowest_price_per_mwh_former_7_mean_gas",
                          "lowest_price_per_mwh_former_7_var_gas":"lowest_price_per_mwh_former_7_var_gas",
                          "highest_price_per_mwh_former_7_mean_gas":"highest_price_per_mwh_former_7_mean_gas",
                          "highest_price_per_mwh_former_7_var_gas":"highest_price_per_mwh_former_7_var_gas"})
    
    
    train['date'] = pd.to_datetime(train['datetime']).dt.date
    # train = train.drop(columns = ['data_block_id'])
    client['date'] =  pd.to_datetime(pd.to_datetime(client['date']) + pd.Timedelta(days=2)).dt.date
    # client = client.drop(columns = ['data_block_id'])
    tmp1 = pd.merge(train, client, how = 'inner', left_on=['date','product_type','county','is_business'], right_on = ['date','product_type','county','is_business'])
    tmp1['datetime'] = pd.to_datetime(tmp1['datetime'])
    tmp1['date'] = pd.to_datetime(tmp1['datetime']).dt.date
    tmp1['date']= pd.to_datetime(tmp1['date'])
    tmp1['hour'] = tmp1['datetime'].dt.hour
    
    print('tmp1的长度')
    print(len(tmp1))
    
    tmp2 = pd.merge(tmp1, fw_loc_groupby, how = 'left', left_on = ['county', 'datetime'], right_on = ['county','forecast_datetime'])
    print('tmp2的长度')
    print(len(tmp2))
    hw_loc_groupby['datetime'] = pd.to_datetime(hw_loc_groupby['datetime'])
    tmp3 = pd.merge(tmp2, hw_loc_groupby, how = 'left', left_on = ['datetime', 'county'], right_on = ['datetime', 'county'])
    print('tmp3的长度')
    print(len(tmp3))
    tmp4 = pd.merge(tmp3, electricity, how = 'left', left_on = ['datetime'], right_on = ['datetime'])
    tmp4['date']= pd.to_datetime(tmp4['date'])
    print('tmp4的长度')
    print(len(tmp4))
    # 合并tmp4和gas表（此时已合并所有train, client, fw, hw, electricity, gas共6个表）
    
    train_all = pd.merge(tmp4, gas, how = 'inner', left_on=['date'],right_on=['date'])
    print('train_all的长度')
    print(len(train_all))
    
    # 新建'week_num'列储存星期信息
    train_all['week_num'] = train_all['datetime'].dt.day_name()
    # 初始化新列is_weekend存储是否为周末
    train_all['is_weekend'] = 0
    # 选择 'week_num' 列中值为 ‘Saturday’ 或 ‘Sunday’ 的行，并将这些行的 'is_weekend' 设置为 1
    train_all.loc[train_all['week_num'].isin(['Saturday', 'Sunday']), 'is_weekend'] = 1

    # 将星期信息进行one-hot编码
    # 隐藏的测试集里面只有Sunday，会导致训练时模型与测试时模型的数据列数不同，产生报错
    # train_all_week_num = pd.get_dummies(train_all['week_num'])
    # train_all = pd.concat([train_all, train_all_week_num], axis = 1)
    train_all = train_all.drop(columns = ['week_num'])
    
    print('添加星期信息后train_all的长度')
    print(len(train_all))
    
    # 新建时间信息
    train_all['dayofyear'] = train_all['datetime'].dt.dayofyear
    train_all['sin(dayofyear)'] = np.sin(np.pi * train_all['dayofyear'] / 183)
    train_all['cos(dayofyear)'] = np.cos(np.pi * train_all['dayofyear'] / 183)
    train_all['sin(hour)'] = np.sin(np.pi * train_all['hour'] / 12)
    train_all['cos(hour)'] = np.cos(np.pi * train_all['hour'] / 12)

    print('添加时间信息后train_all的长度')
    print(len(train_all))
    
    # 添加节日信息
    train_all = add_holidays_as_binary_features(train_all)
    
    print('添加节日信息后train_all的长度')
    print(len(train_all))
    
    #填加target
    target = df_target[['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']].copy()
    target1 = df_target[['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']].copy()
    target['datetime'] = pd.to_datetime(target['datetime'])
    target1['datetime'] = pd.to_datetime(target['datetime'])
    
    target =join_target_on_date(target,target1,1, 1)
    target =join_target_on_date(target,target1,2, 2)
    target =join_target_on_date(target,target1,3, 3)
    target =join_target_on_date(target,target1,4, 4)
    target =join_target_on_date(target,target1,5, 5)
    target =join_target_on_date(target,target1,6, 6)
    target = join_target_on_date(target,target1,7, 7)
    target = target.drop(columns = ['target'])
    
    #合并train_all与target
    train_all = train_all.merge(target, on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
    
    print('添加target信息后train_all的长度')
    print(len(train_all))
    
    # print(train_all[:10000]['datetime'].unique())
    # 最后删除与预测无关的列
    train_all = train_all.drop(columns = ['date', 'datetime', 'forecast_datetime', 'row_id','prediction_unit_id'])
    
    
    
    
    #修改数据类型
    # train_all[['Friday','Monday','Saturday','Sunday','Thursday','Tuesday','Wednesday','country_holiday']] = train_all[['Friday','Monday','Saturday','Sunday','Thursday','Tuesday','Wednesday','country_holiday']].astype(bool)
    
    object_list = ['is_business', 'is_consumption', 'country_holiday']
    for col in object_list:
        if col in train_all.columns:
            train_all[col] = train_all[col].astype(bool)
    
    # train_all.info()
    
    # train_all = train_all.dropna()
    #lightGBM是否可以包含空值
    
    return train_all

In [7]:
DATA_PATH = "/kaggle/input/predict-energy-behavior-of-prosumers"

train = pd.read_csv(f'{DATA_PATH}/train.csv')
gas = pd.read_csv(f'{DATA_PATH}/gas_prices.csv')
client = pd.read_csv(f'{DATA_PATH}/client.csv')
electricity = pd.read_csv(f'{DATA_PATH}/electricity_prices.csv')
hw = pd.read_csv(f'{DATA_PATH}/historical_weather.csv')
fw = pd.read_csv(f'{DATA_PATH}/forecast_weather.csv')
locations =  pd.read_csv(f'{DATA_PATH}/weather_station_to_county_mapping.csv')

# 测试时要求直接输入df_target
df_target = train[['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']]
df_target['is_business'] = df_target['is_business'].astype(bool)
df_target['is_consumption'] = df_target['is_consumption'].astype(bool)
df_target['datetime'] = pd.to_datetime(df_target['datetime'])
# 各个表中的data_block_id列去掉
train = train.drop(columns = ['data_block_id'])
gas = gas.drop(columns = ['data_block_id'])
client = client.drop(columns = ['data_block_id'])
electricity = electricity.drop(columns = ['data_block_id'])
hw = hw.drop(columns = ['data_block_id'])
fw = fw.drop(columns = ['data_block_id'])

/tmp/ipykernel_405/3694854227.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target['is_business'] = df_target['is_business'].astype(bool)
/tmp/ipykernel_405/3694854227.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target['is_consumption'] = df_target['is_consumption'].astype(bool)
/tmp/ipykernel_405/3694854227.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [8]:
train_all = feature_eng1(train, client, gas, electricity, fw, hw, locations, df_target)
train_all = train_all[train_all["target"].notnull()]
train_all

tmp1的长度
2009712
tmp2的长度
2009712
tmp3的长度
2009712
tmp4的长度
2009712
train_all的长度
2009712
添加星期信息后train_all的长度
2009712
添加时间信息后train_all的长度
2009712
添加节日信息后train_all的长度
2009712
添加target信息后train_all的长度
2009712


,county,is_business,product_type,target,is_consumption,eic_count,installed_capacity,hour,hours_ahead_fw,temperature_fw,...,sin(hour),cos(hour),country_holiday,target_1,target_2,target_3,target_4,target_5,target_6,target_7
0,0,False,1,0.793,False,108,952.89,0,34.0,12.774683,...,0.000000,1.000000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,False,1,107.129,True,108,952.89,0,34.0,12.774683,...,0.000000,1.000000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,False,1,2.106,False,108,952.89,1,35.0,12.831120,...,0.258819,0.965926,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,False,1,81.920,True,108,952.89,1,35.0,12.831120,...,0.258819,0.965926,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,False,1,0.955,False,108,952.89,2,36.0,12.782994,...,0.500000,0.866025,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009707,15,True,3,265.328,True,55,2188.20,21,43.0,13.916846,...,-0.707107,0.707107,False,245.319,296.073,146.381,172.973,260.560,266.683,271.112
2009708,15,True,3,0.001,False,55,2188.20,22,44.0,12.470825,...,-0.500000,0.866025,False,0.000,0.000,0.000,0.000,0.101,0.000,0.000
2009709,15,True,3,274.569,True,55,2188.20,22,44.0,12.470825,...,-0.500000,0.866025,False,290.231,299.806,159.128,190.316,238.254,302.144,254.907
2009710,15,True,3,0.000,False,55,2188.20,23,45.0,11.484033,...,-0.258819,0.965926,False,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [9]:
train_all.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2009184 entries, 0 to 2009711
Data columns (total 72 columns):
 #   Column                                 Dtype  
---  ------                                 -----  
 0   county                                 int64  
 1   is_business                            bool   
 2   product_type                           int64  
 3   target                                 float64
 4   is_consumption                         bool   
 5   eic_count                              int64  
 6   installed_capacity                     float64
 7   hour                                   int32  
 8   hours_ahead_fw                         float64
 9   temperature_fw                         float64
 10  dewpoint_fw                            float64
 11  cloudcover_high_fw                     float64
 12  cloudcover_low_fw                      float64
 13  cloudcover_mid_fw                      float64
 14  cloudcover_total_fw                    float64
 15  10_

In [10]:
X_consume = train_all.drop(columns = ['target'])[train_all['is_consumption'] == 1].reset_index(drop = True)
y_consume = train_all[train_all['is_consumption'] == 1]['target'].reset_index(drop = True)
X_produce = train_all.drop(columns = ['target'])[train_all['is_consumption'] == 0].reset_index(drop = True)
y_produce = train_all[train_all['is_consumption'] == 0]['target'].reset_index(drop = True)

In [11]:
from sklearn.model_selection import train_test_split
X_train_consume, X_test_consume, y_train_consume, y_test_consume = train_test_split(X_consume, y_consume,  test_size = 0.2, random_state = 1)
X_train_produce, X_test_produce, y_train_produce, y_test_produce = train_test_split(X_produce, y_produce,  test_size = 0.2, random_state = 1)

In [12]:
#def lgb_objective(trial):
#     params = {
#        
#         'random_state'     : 42,
#         'objective'        : 'l2',
#         "n_estimators"     : trial.suggest_int("n_estimators", 500, 1500),
#         'learning_rate'    : trial.suggest_float('learning_rate', 0.01, 0.1),
#         'num_leaves'       : trial.suggest_int('num_leaves', 20, 1000),
#         'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'colsample_bynode' : trial.suggest_float('colsample_bynode', 0.5, 1.0),
#         'lambda_l1'        : trial.suggest_float('lambda_l1', 1e-2, 10.0),
#         'lambda_l2'        : trial.suggest_float('lambda_l2', 1e-2, 10.0),
#         'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 4, 256),
#         'max_depth'        : trial.suggest_int('max_depth', 5, 10),
#         'max_bin'          : trial.suggest_int('max_bin', 32, 1024),
#     }

     # Fit the model
#     model = lgb.LGBMRegressor(**params)

#     model.fit(X_train_consume, y_train_consume)
    
#     y_pred_consume = model.predict(X_test_consume)

#     return mean_absolute_error(y_test_consume, y_pred_consume)

#study_lgb_consume = optuna.create_study(direction='minimize') 
#study_lgb_consume.optimize(lgb_objective, n_trials=10, show_progress_bar=True)

In [13]:
#def lgb_objective_1(trial):
#     params = {
#        
#         'random_state'     : 42,
#         'objective'        : 'l2',
#         "n_estimators"     : trial.suggest_int("n_estimators", 500, 1500),
#         'learning_rate'    : trial.suggest_float('learning_rate', 0.01, 0.1),
#         'num_leaves'       : trial.suggest_int('num_leaves', 20, 1000),
#         'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
#         'colsample_bynode' : trial.suggest_float('colsample_bynode', 0.5, 1.0),
#         'lambda_l1'        : trial.suggest_float('lambda_l1', 1e-2, 10.0),
#         'lambda_l2'        : trial.suggest_float('lambda_l2', 1e-2, 10.0),
#         'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 4, 256),
#         'max_depth'        : trial.suggest_int('max_depth', 5, 10),
#         'max_bin'          : trial.suggest_int('max_bin', 32, 1024),
#     }
#
     # Fit the model
#     model = lgb.LGBMRegressor(**params)

#     model.fit(X_train_produce, y_train_produce)
    
#     y_pred_produce = model.predict(X_test_produce)

#     return mean_absolute_error(y_test_produce, y_pred_produce)

#study_lgb_produce = optuna.create_study(direction='minimize') 
#study_lgb_produce.optimize(lgb_objective_1, n_trials=10, show_progress_bar=True)

In [14]:
best_params_consume = {'n_estimators': 1431, 'learning_rate': 0.0551787862618339, 'num_leaves': 842, 'colsample_bytree': 0.7041692691308047, 'colsample_bynode': 0.9464257615398879, 'lambda_l1': 3.5553059277534813, 'lambda_l2': 9.699940077832881, 'min_data_in_leaf': 140, 'max_depth': 10, 'max_bin': 942}
best_params_produce = {'n_estimators': 1456, 'learning_rate': 0.04792210443806526, 'num_leaves': 643, 'colsample_bytree': 0.7415703894708183, 'colsample_bynode': 0.6608787398677248, 'lambda_l1': 7.971511344337281, 'lambda_l2': 0.06719129003550212, 'min_data_in_leaf': 44, 'max_depth': 10, 'max_bin': 464}

In [15]:
model_consume = lgb.LGBMRegressor(**best_params_consume)
model_produce = lgb.LGBMRegressor(**best_params_produce)

In [16]:
model_consume.fit(
    X=X_consume,
    y=y_consume
)

model_produce.fit(
    X=X_produce,
    y=y_produce
)

[LightGBM] [Warning] lambda_l1 is set=3.5553059277534813, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.5553059277534813
[LightGBM] [Warning] min_data_in_leaf is set=140, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=140
[LightGBM] [Warning] lambda_l2 is set=9.699940077832881, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.699940077832881


LGBMRegressor(colsample_bynode=0.6608787398677248,
              colsample_bytree=0.7415703894708183, lambda_l1=7.971511344337281,
              lambda_l2=0.06719129003550212, learning_rate=0.04792210443806526,
              max_bin=464, max_depth=10, min_data_in_leaf=44, n_estimators=1456,
              num_leaves=643)

In [17]:
import enefit

env = enefit.make_env()
iter_test = env.iter_test()

In [18]:
# sample_prediction["target"] = 0

In [20]:
for (test, revealed_targets, client, historical_weather,
        forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:
    
    test = test.rename(columns={'prediction_datetime': 'datetime'})
    test = test.drop(columns = ['currently_scored'])
    
    df_forecast       = pd.concat([fw, forecast_weather]).reset_index(drop = True).drop_duplicates()
    df_historical     = pd.concat([hw, historical_weather]).reset_index(drop = True).drop_duplicates()
    revealed_targets = revealed_targets.drop(columns = ['row_id', 'prediction_unit_id'])
    df_target         = pd.concat([df_target, revealed_targets]).reset_index(drop = True).drop_duplicates()
    print(df_target)
    
    X_test = feature_eng1(test, client, gas_prices, electricity_prices, df_forecast, df_historical, locations, df_target)
    
    test['target'] = model_consume.predict(X_test).clip(0)
    test['target_solar'] = model_produce.predict(X_test).clip(0)
    test.loc[test['is_consumption']==0, 'target'] = test.loc[test['is_consumption']==0, 'target_solar']  
    
    sample_prediction['target'] = test['target']
    
    
    # sample_prediction["target"] = model.predict(X_test).clip(0)
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
          target  county  is_business  product_type  is_consumption  \
0          0.713       0        False             1           False   
1         96.590       0        False             1            True   
2          0.000       0        False             2           False   
3         17.314       0        False             2            True   
4          2.904       0        False             3           False   
...          ...     ...          ...           ...             ...   
2018347  197.233      15         True             0            True   
2018348    0.000      15         True             1           False   
2018349   28.404      15         True             1            True   
2018350    0.000      15         True             3           False   
2018351  196.240      15         True             3            True   

                   dat